In [2]:
%pip install whylogs[all] textstat -q


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")

/home/jamie/projects/v1/whylogs/python/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset imdb downloaded and prepared to /home/jamie/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 398.45it/s]


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [15]:
import pandas as pd

df = pd.DataFrame(data=dataset["train"]["text"], columns=["text"])
df.head()

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [17]:
import textstat

# lets look at some scores from textstat,
# nice and numeric so we can just wire these as whylogs UDFs
print(textstat.flesch_kincaid_grade(df["text"][0]))
print(textstat.flesch_kincaid_grade(df["text"][1]))

9.6
10.9


In [24]:
from whylogs.core.datatypes import String
from whylogs.experimental.core.metrics.udf_metric import (
    register_metric_udf,
)

#Here's how you can wire a udf, just add the decorator like these three examples

@register_metric_udf(col_type=String)
def flesch_kincaid_grade(text: str) -> float:
    # TODO: check if input is actually a string, try/except
    return textstat.flesch_kincaid_grade(text)

@register_metric_udf(col_type=String)
def smog_index(text: str) -> float:
    # TODO: check if input is actually a string, try/except
    return textstat.smog_index(text)

@register_metric_udf(col_type=String)
def dale_chall_readability_score(text: str) -> float:
    # TODO: check if input is actually a string, try/except
    return textstat.dale_chall_readability_score(text)



In [25]:
from whylogs.core.resolvers import STANDARD_RESOLVER
from whylogs.core.schema import DeclarativeSchema
from whylogs.experimental.core.metrics.udf_metric import (
    generate_udf_schema,
)

# This is the glue code to wire in any udfs decorated previously
# And this is something Jamie could extend by reading from a metric hub
# but for now we can just decorate a few metrics like the cell above
schema = DeclarativeSchema(
    STANDARD_RESOLVER + generate_udf_schema(),
)

In [27]:
import whylogs as why

# Then loggint the text with the collection of udfs is pretty easy (if slow on 25k rows of text)
results = why.log(df, schema=schema)


In [28]:
# we can upload these to whylabs but lets check if the udfs show up here
results.view().to_pandas()

,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,...,udf/smog_index:distribution/q_90,udf/smog_index:distribution/q_95,udf/smog_index:distribution/q_99,udf/smog_index:distribution/stddev,udf/smog_index:types/boolean,udf/smog_index:types/fractional,udf/smog_index:types/integral,udf/smog_index:types/object,udf/smog_index:types/string,udf/smog_index:types/tensor
column,,,,,,,,,,,,,,,,,,,,,
text,24879.188998,24561.455792,25204.84395,0,25000,0,0,NaN,0.0,None,...,12.9,13.8,15.9,2.25992,0,25000,0,0,0,0
